# Import Library

In [181]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from skimage import io  # Using scikit-image to process image
from enum import Enum
import os


In [262]:
class Config(Enum):
    num_hidden = 1024
    num_component = 2
    learning_rate = 0.1
    random_seed = 0
    epoch = 64


class ImageType(Enum):
    Carambula = 0
    Lychee = 1
    Pear = 2


rng = np.random.default_rng(Config.random_seed.value)
pca = PCA(n_components=Config.num_component.value)


# Read image & reduce dimension

In [183]:
class ImageLoader:
    def __init__(self, dir, split) -> None:
        self.split = split
        self.dir = os.path.join(
            dir, "Data_train" if split == "train" else "Data_test"
        )

        self.labels = np.empty(0, dtype=np.int32)

        for index in ImageType:
            images_path = os.path.join(self.dir, index.name, "*.png")
            images = io.imread_collection(images_path).concatenate() / 255

            try:
                self.images = np.concatenate((self.images, images))
            except AttributeError:
                self.images = images

            self.labels = np.concatenate(
                (
                    self.labels,
                    np.full(images.shape[0], index.value, dtype=np.int32),
                )
            )

        self.images_pca = self.__get_PCA_images()

    def __get_PCA_images(self):
        self.images_reshape = self.images.reshape(self.images.shape[0], -1)

        if self.split == "train":
            return pca.fit_transform(self.images_reshape)
        else:
            return pca.transform(self.images_reshape)

    def get(self):
        return self.images_pca, self.labels


In [184]:
train_x, train_y = ImageLoader(".", "train").get()
test_x, test_y = ImageLoader(".", "test").get()


# NN Model

In [250]:
class NN:
    def __init__(self, size: list) -> None:
        self.size = size
        self.num_layer = len(size)

        self.bias = [rng.random((i, 1)) for i in size[1:]]
        self.weights = [rng.random((j, k)) for k, j in zip(size[:-1], size[1:])]

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))

    def softmax(self, x):
        return np.exp(x) / sum(np.exp(x))

    def cost_derivative(self, label, pred):
        cost = 0
        for i, output in enumerate(pred):
            if i == label:
                cost += output - 1
            else:
                cost += output
        return cost

    def dim_increase(self, x):
        return x.reshape(-1, 1)

    def __feed_forward(self, feature: np.ndarray) -> np.ndarray:
        # Initial declare the variable
        activation = feature
        activations = [activation]
        zs = []

        # Hidden layer
        for b, w in zip(self.bias, self.weights):
            z = np.dot(w, activation) + b
            activation = self.sigmoid(z)

            zs.append(z)
            activations.append(activation)

        # Output layer
        activations[-1] = self.softmax(activations[-1])

        return zs, activations

    def __back_propagation(self, feature, label, learning_rate):
        zs, activations = self.__feed_forward(feature)
        weight_gradient = [np.zeros(w.shape) for w in self.weights]
        bias_gradient = [np.zeros(b.shape) for b in self.bias]

        delta = self.cost_derivative(
            label, activations[-1]
        ) * self.sigmoid_derivative(zs[-1])

        bias_gradient[-1] = delta
        weight_gradient[-1] = np.dot(delta, activations[-2].T)

        for l in range(2, self.num_layer):
            delta = np.dot(
                self.weights[-l + 1].T, delta
            ) * self.sigmoid_derivative(zs[-l])

            bias_gradient[-l] = delta
            weight_gradient[-l] = np.dot(delta, activations[-l - 1].T)

        return bias_gradient, weight_gradient

    def train(self, features, labels, learning_rate, epoch):
        for _ in range(epoch):
            features_shuffle = rng.permutation(features)
            for label, feature in zip(labels, features_shuffle):
                bias_gradient, weight_gradient = self.__back_propagation(
                    self.dim_increase(feature), label, learning_rate
                )

                self.bias = [
                    b - learning_rate * b_gradient
                    for b, b_gradient in zip(self.bias, bias_gradient)
                ]

                self.weights = [
                    w - learning_rate * w_gradient
                    for w, w_gradient in zip(self.weights, weight_gradient)
                ]

    def pred(self, test_features):
        test_result = [
            np.argmax(self.__feed_forward(self.dim_increase(feature))[1][-1])
            for feature in test_features
        ]

        return test_result


In [268]:
model = NN(
    [Config.num_component.value, Config.num_hidden.value, len(ImageType)]
)
model.train(train_x, train_y, Config.learning_rate.value, Config.epoch.value)
pred_y = model.pred(test_x)


In [270]:
def evaluate(pred_y, test_y):
    corr = 0
    for pred, test in zip(pred_y, test_y):
        if pred == test:
            corr += 1

    return corr / len(pred_y)


print(evaluate(pred_y, test_y))


0.3333333333333333


In [209]:
def rng_int(seed):
    rng = np.random.default_rng(seed)
    return rng.integers(100)


In [215]:
rng_int(100)


76